# Colab

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install ultralytics
clear_output()

# Imports

In [7]:
import os
import json
import math
import random
import yaml
import shutil
from shutil import copyfile
import time
from PIL import Image, ImageDraw

from ultralytics import YOLO

# Convert `JSON` --> `YOLO-OBB`

In [6]:
# Скрипт конвертации данных экспорта из Lable Studio в формат для YOLO версий 8 и выше

# Путь к папке с проектами Label Studio
# project_path = '/content/drive/MyDrive/Vtormet/dataset/' # Colab папки со структурой: папка images + файл JSON-MIN
project_path = "data/raw/"  #  VSC папки со структурой: папка images + файл JSON-MIN

# Путь к папке с результатами конвертации
# output_path = "/content/drive/MyDrive/Vtormet/output/" # Colab
output_path = "data/output/"  # VSC

# Создаем папки для размещения результатов конвертации
if not os.path.exists(output_path):
    os.makedirs(output_path)
images_path = os.path.join(output_path, "images")
os.makedirs(images_path, exist_ok=True)
labels_path = os.path.join(output_path, "labels")
os.makedirs(labels_path, exist_ok=True)
train_images_path = os.path.join(images_path, "train")
os.makedirs(train_images_path, exist_ok=True)
train_labels_path = os.path.join(labels_path, "train")
os.makedirs(train_labels_path, exist_ok=True)
val_images_path = os.path.join(images_path, "val")
os.makedirs(val_images_path, exist_ok=True)
val_labels_path = os.path.join(labels_path, "val")
os.makedirs(val_labels_path, exist_ok=True)

# Создаем словарь для сопоставления имен классов с их индексами
classes = {}

# Перебираем все проекты Label Studio
all_images = []
all_labels = []

for project in os.listdir(project_path):
    project_dir = os.path.join(project_path, project)
    if os.path.isdir(project_dir):
        # Ищем файл json с разметкой
        json_files = [f for f in os.listdir(project_dir) if f.endswith(".json")]
        if json_files:
            json_file = json_files[0]
            with open(os.path.join(project_dir, json_file)) as f:
                data = json.load(f)

            # Создаем словарь для сопоставления имен классов с их индексами
            for label in data[0]["label"][0]["rectanglelabels"]:
                if label not in classes:
                    classes[label] = len(classes)

            # Создаем папку для текстовых файлов разметки
            labels_dir = os.path.join(project_dir, "labels")
            if not os.path.exists(labels_dir):
                os.makedirs(labels_dir)

            # Перебираем все изображения в проекте
            for image in data:
                # Получаем имя файла изображения
                filename = os.path.basename(image["image"])
                # Получаем путь к файлу изображения
                image_path = os.path.join(project_dir, "images", filename)
                if not os.path.exists(image_path):
                    print(f"Изображение {image_path} не найдено.")
                    continue

                # Получаем путь к текстовому файлу разметки
                label_path = os.path.join(
                    labels_dir, f"{os.path.splitext(filename)[0]}.txt"
                )

                # Создаем текстовый файл разметки
                with open(label_path, "w") as f:
                    # Перебираем все бэндбоксы на изображении
                    for box in image["label"]:
                        # Получаем значения размера изображения
                        width, height = box["original_width"], box["original_height"]
                        # Получаем координаты бэндбокса (относительные)
                        x, y, w, h = (
                            box["x"] / 100 * width,
                            box["y"] / 100 * height,
                            box["width"] / 100 * width,
                            box["height"] / 100 * height,
                        )
                        # Получаем угол поворота бэндбокса
                        angle = box.get("rotation", 0)  # значение по умолчанию 0
                        # Получаем класс объекта
                        class_name = box["rectanglelabels"][0]
                        # Получаем индекс класса объекта
                        class_index = classes[class_name]

                        # Преобразование координат с учетом угла поворота
                        x1 = x
                        y1 = y
                        x2 = x + w * math.cos(angle * math.pi / 180)
                        y2 = y + w * math.sin(angle * math.pi / 180)
                        x3 = (
                            x
                            - h * math.sin(angle * math.pi / 180)
                            + w * math.cos(angle * math.pi / 180)
                        )
                        y3 = (
                            y
                            + h * math.cos(angle * math.pi / 180)
                            + w * math.sin(angle * math.pi / 180)
                        )
                        x4 = x - h * math.sin(angle * math.pi / 180)
                        y4 = y + h * math.cos(angle * math.pi / 180)

                        # Записываем информацию о бэндбоксе и классе с нормализованными координатами:
                        f.write(
                            f"{class_index} {x1/width} {y1/height} {x2/width} {y2/height} {x3/width} {y3/height} {x4/width} {y4/height}\n"
                        )

                all_images.append(image_path)
                all_labels.append(label_path)

# Перемешиваем списки и разделяем на тренировочную и валидационную выборки
combined = list(zip(all_images, all_labels))
random.shuffle(combined)
train_split = int(0.8 * len(combined))
train_files = combined[:train_split]
val_files = combined[train_split:]

# Копируем файлы в соответствующие директории
for image_path, label_path in train_files:
    copyfile(image_path, os.path.join(train_images_path, os.path.basename(image_path)))
    copyfile(label_path, os.path.join(train_labels_path, os.path.basename(label_path)))

for image_path, label_path in val_files:
    copyfile(image_path, os.path.join(val_images_path, os.path.basename(image_path)))
    copyfile(label_path, os.path.join(val_labels_path, os.path.basename(label_path)))

# Создаем файл yaml с данными для YOLO по разметке
data_yaml = {
    "train": os.path.join(images_path, "train"),
    "val": os.path.join(images_path, "val"),
    "names": list(classes.keys()),
    "nc": len(classes),
}

with open(os.path.join(output_path, "data.yaml"), "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False, allow_unicode=True)

# Выводим количество файлов в тренировочной и валидационной выборке
print(f"Файл data.yaml сформирован")
print(f"Тренировочная выборка: {len(train_files)} изображений и меток")
print(f"Валидационная выборка: {len(val_files)} изображений и меток")

Файл data.yaml сформирован
Тренировочная выборка: 65 изображений и меток
Валидационная выборка: 17 изображений и меток


## Check

In [11]:
# Код проверки (предложил Герасимов Алексей):
# Берём картинку и рисуем на ней OBB для проверки

# Открываем изображение
# image_path = '/content/drive/MyDrive/Vtormet/output/images/train/693c1a4c-1.jpg'
image_path = "data/output/images/train/0b4a7ca9-DSCF1233.JPG"
image = Image.open(image_path)

# Создаем объект ImageDraw
draw = ImageDraw.Draw(image)

# Открываем файл с координатами
# coordinates_path = '/content/drive/MyDrive/Vtormet/output/labels/train/693c1a4c-1.txt'
coordinates_path = "data/output/labels/train/0b4a7ca9-DSCF1233.txt"
with open(coordinates_path, "r") as f:
    lines = f.readlines()

# Читаем координаты из файла и рисуем четырехугольники
width, height = image.size
for line in lines:
    line_parts = line.strip().split(" ")
    class_id = int(line_parts[0])
    x1, y1 = float(line_parts[1]) * width, float(line_parts[2]) * height
    x2, y2 = float(line_parts[3]) * width, float(line_parts[4]) * height
    x3, y3 = float(line_parts[5]) * width, float(line_parts[6]) * height
    x4, y4 = float(line_parts[7]) * width, float(line_parts[8]) * height

    # Рисуем четырехугольник
    draw.polygon(
        [(x1, y1), (x2, y2), (x3, y3), (x4, y4)], outline="red", fill=None, width=3
    )

# Сохраняем изображение с нарисованным четырехугольником
output_path = "data/test/test_693c1a4c-1.jpg"
image.save(output_path)